In [ ]:
%matplotlib inline


Mạng Neural
===============

Khi sử dụng pytorch, package ``torch.nn`` sẽ giúp bạn xây dựng một mạng neural.

Trong bài trước, ta đã tìm hiểu về package ``autograd`` - tính đạo hàm tự động, package ``nn`` sẽ phụ thuộc vào ``autograd`` để định nghĩa mô hình và tính đạo hàm.

Để định nghĩa một mô hình mạng Neural, bạn định nghĩa một ``nn.Module`` gồm các lớp của mạng Neural và thủ tục ``forward(input)`` mô tả quá trình tính toán trên dữ liệu vào và trả về ``output``.

Để ví dụ hãy xem xét mạng neural dùng để phân loại chữ số này:

.. figure:: /_static/img/mnist.png
   :alt: convnet

   convnet

Đây là một mạng neural đơn giản mà dữ liệu chỉ tiến về phía trước (feed-forward): dữ liệu đầu vào (hình ảnh chữ số) được đi qua từng lớp trên mạng, và cuối cùng tính ra kết quả (là xác suất để ảnh là chữ số từ 0 đến 9).

Quá trình huấn luyện một mạng neural như vậy thường bao gồm các bước sau:

- Định nghĩa mạng neural với các tham số cần học (hay còn gọi là trọng số/weights)
- Lặp qua tập hợp các dữ liệu đầu vào
    - Cho mạng tính kết quả dựa trên dữ liệu đầu vào
    - Tính độ lệch của kết quả với kết quả mong đợi (còn gọi là hàm mất mát / loss function)
    - Tính đạo hàm/gradient (của hàm mất mát) trên các trọng số của mạng (bước này còn gọi là lan truyền ngược / backpropgation bởi vì việc tính đạo hàm sẽ diễn ra bằng cách lần ngược từng lớp của mạng)
    - Cập nhật giá trị mới của trọng số của mạng; ví dụ dùng công thức đơn giản sau: ``weight = weight - learning_rate * gradient``

Như trong bài trước về ``autograd``, ta thấy sức mạnh của ``autograd`` sẽ thể hiện ở chỗ, bạn chỉ cần định nghĩa mạng Neural với các lớp và hàm ``forward``, việc tính đạo hàm bằng lan truyền ngược (hàm ``backward``) sẽ được định nghĩa tự động bởi ``autograd``.


Định nghĩa mạng
------------------

Giờ ta hãy định nghĩa mạng trên đây bằng pytorch

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # Đầu vào: 1 channel ảnh, đầu ra: 6 channels, convolution: vuông 5x5
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # Lớp biến đổi tuyến tính / kết nối hoàn toàn (fully connected): y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max-pool trên cửa sổ kích thước 2x2
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # Nếu cửa sổ là hình vuông bạn cũng có thể chỉ ghi kích thước một cạnh
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # Loại trừ đi chiều đầu tiên (kích thước của loạt dữ liệu / batch)
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


Như đã đề cập ở trên, bạn chỉ cần định nghĩa hàm tiến ``forward``, và hàm lan truyền ngược ``backward``
(để tính đạo hàm) sẽ được ``autograd`` tự định nghĩa giúp bạn.
Trong hàm ``forward`` bạn có thể sử dụng bất kỳ phép tính nào trên ``Tensor``.

Các trọng số cần học của mô hình sẽ được trả về qua ``net.parameters()``.

In [4]:
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight

10
torch.Size([6, 1, 5, 5])


Let try a random 32x32 input
Note: Expected input size to this net(LeNet) is 32x32. To use this net on
MNIST dataset, please resize the images from the dataset to 32x32.



In [ ]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

Zero the gradient buffers of all parameters and backprops with random
gradients:



In [ ]:
net.zero_grad()
out.backward(torch.randn(1, 10))

<div class="alert alert-info"><h4>Note</h4><p>``torch.nn`` only supports mini-batches. The entire ``torch.nn``
    package only supports inputs that are a mini-batch of samples, and not
    a single sample.

    For example, ``nn.Conv2d`` will take in a 4D Tensor of
    ``nSamples x nChannels x Height x Width``.

    If you have a single sample, just use ``input.unsqueeze(0)`` to add
    a fake batch dimension.</p></div>

Before proceeding further, let's recap all the classes you’ve seen so far.

**Recap:**
  -  ``torch.Tensor`` - A *multi-dimensional array* with support for autograd
     operations like ``backward()``. Also *holds the gradient* w.r.t. the
     tensor.
  -  ``nn.Module`` - Neural network module. *Convenient way of
     encapsulating parameters*, with helpers for moving them to GPU,
     exporting, loading, etc.
  -  ``nn.Parameter`` - A kind of Tensor, that is *automatically
     registered as a parameter when assigned as an attribute to a*
     ``Module``.
  -  ``autograd.Function`` - Implements *forward and backward definitions
     of an autograd operation*. Every ``Tensor`` operation, creates at
     least a single ``Function`` node, that connects to functions that
     created a ``Tensor`` and *encodes its history*.

**At this point, we covered:**
  -  Defining a neural network
  -  Processing inputs and calling backward

**Still Left:**
  -  Computing the loss
  -  Updating the weights of the network

Loss Function
-------------
A loss function takes the (output, target) pair of inputs, and computes a
value that estimates how far away the output is from the target.

There are several different
`loss functions <https://pytorch.org/docs/nn.html#loss-functions>`_ under the
nn package .
A simple loss is: ``nn.MSELoss`` which computes the mean-squared error
between the input and the target.

For example:



In [ ]:
output = net(input)
target = torch.randn(10)  # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

Now, if you follow ``loss`` in the backward direction, using its
``.grad_fn`` attribute, you will see a graph of computations that looks
like this:

::

    input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d
          -> view -> linear -> relu -> linear -> relu -> linear
          -> MSELoss
          -> loss

So, when we call ``loss.backward()``, the whole graph is differentiated
w.r.t. the loss, and all Tensors in the graph that has ``requires_grad=True``
will have their ``.grad`` Tensor accumulated with the gradient.

For illustration, let us follow a few steps backward:



In [ ]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

Backprop
--------
To backpropagate the error all we have to do is to ``loss.backward()``.
You need to clear the existing gradients though, else gradients will be
accumulated to existing gradients.


Now we shall call ``loss.backward()``, and have a look at conv1's bias
gradients before and after the backward.



In [ ]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

Now, we have seen how to use loss functions.

**Read Later:**

  The neural network package contains various modules and loss functions
  that form the building blocks of deep neural networks. A full list with
  documentation is `here <https://pytorch.org/docs/nn>`_.

**The only thing left to learn is:**

  - Updating the weights of the network

Update the weights
------------------
The simplest update rule used in practice is the Stochastic Gradient
Descent (SGD):

     ``weight = weight - learning_rate * gradient``

We can implement this using simple python code:

.. code:: python

    learning_rate = 0.01
    for f in net.parameters():
        f.data.sub_(f.grad.data * learning_rate)

However, as you use neural networks, you want to use various different
update rules such as SGD, Nesterov-SGD, Adam, RMSProp, etc.
To enable this, we built a small package: ``torch.optim`` that
implements all these methods. Using it is very simple:



In [ ]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update

.. Note::

      Observe how gradient buffers had to be manually set to zero using
      ``optimizer.zero_grad()``. This is because gradients are accumulated
      as explained in `Backprop`_ section.

